# SQL TEST

## Package Installation
To finish this test, you need to install pandasql package, which can be installed by executing next cell

### Package/Data import

In [23]:
# Install pandasql package. Run this once, onces installation is successful, you can block this cell and do not run it again.
!pip install pandasql

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [24]:
import pandas as pd
import pandasql as pdql


In [25]:
cust = pd.read_csv('InterviewCSVs/Customers.csv',encoding="latin-1")
categories = pd.read_csv('InterviewCSVs/Categories.csv',encoding="latin-1")
emp = pd.read_csv('InterviewCSVs/Employees.csv',encoding="latin-1")
ord = pd.read_csv('InterviewCSVs/Orders.csv',encoding="latin-1")
ord_dtl = pd.read_csv('InterviewCSVs/OrderDetails.csv',encoding="latin-1")
prod = pd.read_csv('InterviewCSVs/Products.csv',encoding="latin-1")
shippers = pd.read_csv('InterviewCSVs/Shippers.csv',encoding="latin-1")
suppliers = pd.read_csv('InterviewCSVs/Suppliers.csv',encoding="latin-1")

### Data Model
![Data Model](InterviewCSVs\DataModel.PNG)

### Query Function
To simplify the process, make a function

In [26]:
pysql = lambda q: pdql.sqldf(q, globals())

## As an example, we do answer Q1 query for you. Follow the same pattern and answer the rest.
### Q1: List all customers from country of "Canada"

In [27]:
query = '''
        select 
            CustomerID
            , CustomerName
            , PostalCode           
        from cust            
        where cust.Country = 'Canada';
        '''
pysql(query)

,CustomerID,CustomerName,PostalCode
0,10,Bottom-Dollar Marketse,T2F 8M4
1,42,Laughing Bacchus Wine Cellars,V3F 2K1
2,51,Mère Paillarde,H1J 1C3


### Q2: List all orders from customers of "France"

In [28]:
query = '''
        select
            o.OrderID,
            o.CustomerID,
            o.EmployeeID,
            o.OrderDate,
            o.ShipperID
    from
        ord as o
        
    join 
        cust as c
    
        on c.CustomerID = o.CustomerID
        where c.Country = 'France';
       
        '''
pysql(query)

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10265,7,2,1996-07-25,1
1,10297,7,5,1996-09-04,2
2,10360,7,4,1996-11-22,3
3,10436,7,3,1997-02-05,2
4,10331,9,9,1996-10-16,1
5,10340,9,1,1996-10-29,3
6,10362,9,3,1996-11-25,1
7,10311,18,1,1996-09-20,3
8,10408,23,8,1997-01-08,1
9,10350,41,6,1996-11-11,2


### Q3: How many customers are from contry of "Mexico"? 

In [29]:
query = '''

    select count(CustomerName)
    
    from cust as c
    
    where c.Country='Mexico';

'''
pysql(query)

,count(CustomerName)
0,5


### Q4: What is the total ordered price for category of "Beverages"?

In [30]:
query = '''
        select sum(p.Price)
        
        from
            prod as p
            
        join 
            categories as c
        
        on c.CategoryID = p.CategoryID
        
        where c.CategoryName = 'Beverages';

'''
pysql(query)

,sum(p.Price)
0,455.75


### Q5: What is the most recent order in the database?

In [31]:
query = '''
    select * from 
    
        ord
        
    order by OrderDate DESC
    
    LIMIT 1;
        
        '''
pysql(query)

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10443,66,8,1997-02-12,1


### Q6: What is the total dollar amount sold together in 1997? 

In [32]:
query = '''

    SELECT sum(p.Price)
       
      FROM prod as p
  
      JOIN ord_dtl as od
      
          ON p.ProductID = od.ProductID 
      
      JOIN ord as o
      
          ON o.OrderID = od.OrderID
  
      WHERE strftime('%Y',o.OrderDate)='1997';
        
'''
pysql(query)

,sum(p.Price)
0,3364.79


### Q7: What are the suppliers and categories of top 5 selling products sold in 1996?

In [33]:
query = '''


SELECT s.SupplierID, s.SupplierName, 

    c.CategoryID, c.CategoryName, c.Description,
    
    od.Quantity, o.OrderDate
       
      FROM suppliers as s
      
      JOIN prod as p
            
            ON s.SupplierID = p.SupplierID
    
      JOIN categories as c
            
            ON c.CategoryID = p.CategoryID
    
  
      JOIN ord_dtl as od
      
            ON p.ProductID = od.ProductID 
      
      JOIN ord as o
      
            ON o.OrderID = od.OrderID
          
      
  
      WHERE strftime('%Y',o.OrderDate)='1996'
      
      ORDER BY od.Quantity DESC
      
      LIMIT 5;



'''
pysql(query)

,SupplierID,SupplierName,CategoryID,CategoryName,Description,Quantity,OrderDate
0,25,Ma Maison,6,Meat/Poultry,Prepared meats,120,1996-12-30
1,16,Bigfoot Breweries,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",100,1996-08-21
2,7,"Pavlova, Ltd.",2,Condiments,"Sweet and savory sauces, relishes, spreads, an...",80,1996-10-08
3,8,"Specialty Biscuits, Ltd.",3,Confections,"Desserts, candies, and sweet breads",80,1996-11-04
4,28,Gai pâturage,4,Dairy Products,Cheeses,80,1996-11-21


### Q8: Write a query that lists all customers and shippers managed by Steven Buchanan.


In [34]:
query = '''
    
    select c.CustomerID, c.CustomerName,
    
        s.ShipperID, s.ShipperName,
        
        e.FirstName, e.LastName
        
        from cust as c
        
            join ord as o
            
                on c.CustomerID=o.CustomerID
                
            join emp as e
            
                on o.EmployeeID=e.EmployeeID
                
            join shippers as s
            
                on o.ShipperID= s.ShipperID
                
             where e.FirstName='Steven' AND e.LastName='Buchanan';
    

'''
pysql(query)

,CustomerID,CustomerName,ShipperID,ShipperName,FirstName,LastName
0,7,Blondel père et fils,2,United Package,Steven,Buchanan
1,14,Chop-suey Chinese,2,United Package,Steven,Buchanan
2,24,Folk och fä HB,3,Federal Shipping,Steven,Buchanan
3,41,La maison d'Asie,1,Speedy Express,Steven,Buchanan
4,60,Princesa Isabel Vinhoss,1,Speedy Express,Steven,Buchanan
5,62,Queen Cozinha,2,United Package,Steven,Buchanan
6,72,Seven Seas Imports,3,Federal Shipping,Steven,Buchanan
7,87,Wartian Herkku,3,Federal Shipping,Steven,Buchanan
8,87,Wartian Herkku,3,Federal Shipping,Steven,Buchanan
9,89,White Clover Markets,1,Speedy Express,Steven,Buchanan


### Q9: Write a SQL query which returns a single aggregate metric indicating how valuable a customer is to the company for a given week. What assumptions did you make?

In [35]:
#Assumption: Rank customers based on total value of purchased goods in a particular week. 
#Rank 1 ...n : Rank 1 means most valuable customer for that week
#Week numbers in SQLite are from 00-53 ->ref: https://sqlite.org/lang_datefunc.html

query = '''

select cust.CustomerID, cust.CustomerName, sum(prod.Price*ord_dtl.Quantity)  AS price,
strftime('%W',ord.OrderDate) as week_no,
dense_rank() OVER (PARTITION BY strftime('%W',ord.OrderDate)
   ORDER BY sum(prod.Price*ord_dtl.Quantity) DESC ) AS 'rank'
from cust
    join ord
        on cust.CustomerID=ord.CustomerID
    join ord_dtl
        on ord.OrderID=ord_dtl.OrderID
    join prod
        on ord_dtl.ProductID=prod.ProductID
group by cust.CustomerName, strftime('%W',ord.OrderDate)
order by strftime('%W',ord.OrderDate) ASC,
sum(prod.Price*ord_dtl.Quantity) DESC;
'''

pysql(query)


,CustomerID,CustomerName,price,week_no,rank
0,65,Rattlesnake Canyon Grocery,4837.02,00,1
1,20,Ernst Handel,4652.45,00,2
2,19,Eastern Connection,3829.59,00,3
3,49,Magazzini Alimentari Riuniti,2096.90,00,4
4,62,Queen Cozinha,2527.00,01,1
...,...,...,...,...,...
181,60,Princesa Isabel Vinhoss,1054.00,52,6
182,36,Hungry Coyote Import Store,553.00,52,7
183,17,Drachenblut Delikatessend,108.00,52,8
184,71,Save-a-lot Markets,3420.00,53,1


### Q10: Write a query on CustomerName column in cust table to change all French names to English by converting French letters to the nearest English. For instance 'è' can be changed to 'e'.

In [36]:
#Since the cust table doesn't exist, and the table can't obviously be altered/modified/changed
#Did a little work around and modified the dataframe

#Replace all compatibility characters with their equivalents
#Encode to ASCII which uses only english characters, and then decode back to latin-1
cust['CustomerName'] = cust['CustomerName'].str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('latin-1')

query = '''
select CustomerName from cust;
'''
pysql(query)



,CustomerName
0,Alfreds Futterkiste
1,Ana Trujillo Emparedados y helados
2,Antonio Moreno Taqueria
3,Around the Horn
4,Berglunds snabbkop
...,...
86,Wartian Herkku
87,Wellington Importadora
88,White Clover Markets
89,Wilman Kala


### Q11: Write a query to identify the top 5 countries buying 'Aniseed Syrup'.

In [37]:
query = '''

    select c.Country, od.Quantity, p.ProductName
    
        from cust as c
        
            join ord as o
            
                on c.CustomerID=o.CustomerID
                
            join ord_dtl as od
            
                on o.OrderID=od.OrderID
                
            join prod as p
            
                on od.ProductID=p.ProductID
                
            WHERE p.ProductName='Aniseed Syrup'
            
                Order by od.Quantity DESC
                
                LIMIT 5;
'''
pysql(query)

,Country,Quantity,ProductName
0,Venezuela,50,Aniseed Syrup
1,UK,30,Aniseed Syrup
